In [164]:
import pandas as pd
import numpy as np
import plotly as px
from colorama import Fore, Back, Style
import json

In [144]:
# Couleurs diverses
VERT='\033[0;32m' # vert mince
BLANC='\033[1;37m' # blanc gras
DEFAUT='\033[0;m' # couleur console par defaut
NOIR='\033[0;30m' # noir mince
ROUGE='\033[0;31m' # rouge mince
MARRON='\033[0;33m' # marron mince
BLEU='\033[0;34m' # bleu fonce mince
VIOLET='\033[0;35m' # violet mince
CYAN='\033[0;36m' # cyan mince
GRIS='\033[0;37m' # gris clair mince
BLANCLEGER='\033[0;38m' # blanc mince
ROUGECLAIR='\033[1;31m' # rouge clair gras
VERTCLAIR='\033[1;32m' # vert clair gras
JAUNE='\033[1;33m' # jaune gras
BLEUCLAIR='\033[1;34m' # bleu clair gras
ROSE='\033[1;35m' # rose clair gras
CYANCLAIR='\033[1;36m' # cyan clair gras

# I - Import data

In [145]:
# This csv is imported with a value added in the last ligne
# There was originally a missing value
# Here in a csv : In last rows / col 1 : 30032 was added
file_path = '../data/ESP_PUBLIC.IDENTITE_ARBRE.csv'
df = pd.read_csv(file_path, sep=',', header = 0, index_col=False)
dff = df

# II - Computations

##### Missing years for plantation

In [146]:
# Convert the column year from FLOAT --> INT
# --> Not working because of Nan : dff["ANNEEDEPLANTATION"].astype("int")
missingYear = dff["ANNEEDEPLANTATION"].isnull().value_counts()[1]
numberOfTree = dff["ANNEEDEPLANTATION"].size

print(CYANCLAIR, "Total of Trees -->", BLEUCLAIR, numberOfTree, \
      CYANCLAIR, "<-- with total of -->", BLEUCLAIR, missingYear,\
      CYANCLAIR, "<-- missing years")

 Total of Trees -->  31619  <-- with total of -->  1438  <-- missing years


##### Total of trees for specific year

In [153]:
# First replace the .nan by float(0)
# After that convert float to int
dff["ANNEEDEPLANTATION"].fillna(0).astype("int")

0        1977
1        2001
2        1975
3        1978
4        1960
         ... 
31614    2014
31615    2014
31616    2014
31617    2014
31618    1995
Name: ANNEEDEPLANTATION, Length: 31619, dtype: int64

In [152]:
firstYearPlanted = int(dff["ANNEEDEPLANTATION"].min())
lastYearPlanted = int(dff["ANNEEDEPLANTATION"].max())
print(CYANCLAIR, "The first tree was planted in -->", BLEUCLAIR, firstYearPlanted)
print(CYANCLAIR, "The last tree was planted in --->", BLEUCLAIR, lastYearPlanted)

 The first tree was planted in -->  1900
 The last tree was planted in --->  2021


##### Study by years

In [158]:
df_years = dff[["ANNEEDEPLANTATION"]]
df_years.value_counts()

ANNEEDEPLANTATION
1975.0               3631
2001.0               1488
1980.0               1445
1970.0               1366
2015.0               1012
                     ... 
1959.0                 16
1924.0                 11
1964.0                  6
1900.0                  4
2021.0                  3
Length: 65, dtype: int64

Depuis 2014	M. Eric PIOLLE
De 1995 à 2014	M. Michel DESTOT
De 1983 à 1995	M. Alain CARIGNON
De mars 1965 à 1983	M. Hubert DUBEDOUT
De 1959 à mars 1965	M. Albert MICHALLON
Du 23 janvier 1949 à 1959	M. Léon MARTIN
Du 20 décembre 1948 au 23 janvier 1949	M. Rémy PERINETTI
De 1947 au 20 décembre 1948	M. Marius BALLY
De avril 1945 à 1947	M. Léon MARTIN
Du 23 août 1944 à avril 1945	M. Frédéric LAFLEUR
Du 12 mai 1935 au 23 août 1944	M. Paul COCAT
De 1932 au 12 mai 1935	M. Léon MARTIN
Du 11 décembre 1919 à 1932	M. Paul MISTRAL
De mai 1910 au 11 décembre 1919	M. Nestor CORNIER
De 1908 à mai 1910	M. Félix VIALLET
De 1904 à 1908	M. Charles RIVAIL
Du 4 novembre 1896 à 1904	M. Stéphane JAY

In [161]:
mairesDict = {
    "Stephane JAY" : [1896, 1904],
    "Charles RIVAIL" : [1904, 1908],
    "Felix VIALLET" : [1908, 1910],
    "Nestor CORNIER" : [1910, 1919],
    "Paul MISTRAL" : [1919, 1932],
    "Leon MARTIN" : [1932, 1935],
    "Paul COCAT" : [1935, 1944],
    "Fredric LAFLEUR" : [1944, 1945],
    "Léon MARTIN" : [1945, 1947],
    "Marius BALLY" : [1947, 1948],
    "Rémy PERINETTI" : [1948, 1949],
    "Léon MARTIN" : [1949, 1959],
    "Albert MICHALLON" : [1959, 1965],
    "Hubert DUBEDOUT" : [1965, 1983],
    "Alain CARIGNON" : [1983, 1995],
    "Michel DESTOT" : [1995, 2014],
    "Eric PIOLLE" : [2014, 2021],
}

In [165]:
print(json.dumps(mairesDict, indent=2))

{
  "St\u00e9phane JAY": [
    1896,
    1904
  ],
  "Charles RIVAIL": [
    1904,
    1908
  ],
  "F\u00e9lix VIALLET": [
    1908,
    1910
  ],
  "Nestor CORNIER": [
    1910,
    1919
  ],
  "Paul MISTRAL": [
    1919,
    1932
  ],
  "L\u00e9on MARTIN": [
    1949,
    1959
  ],
  "Paul COCAT": [
    1935,
    1944
  ],
  "Fr\u00e9d\u00e9ric LAFLEUR": [
    1944,
    1945
  ],
  "Marius BALLY": [
    1947,
    1948
  ],
  "R\u00e9my PERINETTI": [
    1948,
    1949
  ],
  "Albert MICHALLON": [
    1959,
    1965
  ],
  "Hubert DUBEDOUT": [
    1965,
    1983
  ],
  "Alain CARIGNON": [
    1983,
    1995
  ],
  "Michel DESTOT": [
    1995,
    2014
  ],
  "Eric PIOLLE": [
    2014,
    2021
  ]
}
